In [1]:
import pandas as pd
import string
import nltk
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,HashingVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
import re

In [2]:
df=pd.read_csv('spam.csv',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df.shape

(5572, 5)

In [4]:
df.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [5]:
df['Unnamed: 4'].unique().size

6

In [6]:
df.drop_duplicates()
df.shape

(5572, 5)

In [7]:
df['length']=df['v2'].str.len()
df.tail(20)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,length
5552,ham,Have a safe trip to Nigeria. Wish you happines...,NaN,NaN,NaN,91
5553,ham,Hahaha..use your brain dear,NaN,NaN,NaN,27
5554,ham,Well keep in mind I've only got enough gas for...,NaN,NaN,NaN,98
5555,ham,Yeh. Indians was nice. Tho it did kane me off ...,NaN,NaN,NaN,153
5556,ham,Yes i have. So that's why u texted. Pshew...mi...,NaN,NaN,NaN,63
5557,ham,No. I meant the calculation is the same. That ...,NaN,NaN,NaN,273
5558,ham,"Sorry, I'll call later",NaN,NaN,NaN,22
5559,ham,if you aren't here in the next &lt;#&gt; hou...,NaN,NaN,NaN,66
5560,ham,Anything lor. Juz both of us lor.,NaN,NaN,NaN,33
5561,ham,Get me out of this dump heap. My mom decided t...,NaN,NaN,NaN,70


In [8]:
stop_words=set(stopwords.words() + ['u','U','dun','R','wif','dat'])

In [11]:
df['v2']=df['v2'].apply(lambda x: ' '.join(t for t in x.split() if t not in stop_words))
df['v2']=df['v2'].apply(lambda x: re.sub('[^a-zA-Z]',' ', x))


df['clean_length']=df['v2'].str.len()
df.head()


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,length,clean_length
0,ham,Go jurong point crazy Available bugis great...,NaN,NaN,NaN,111,80
1,ham,Ok lar Joking oni,NaN,NaN,NaN,29,23
2,spam,Free entry wkly comp win FA Cup final tkts ...,NaN,NaN,NaN,155,141
3,ham,early hor say,NaN,NaN,NaN,49,19
4,ham,Nah I usf lives,NaN,NaN,NaN,61,16


In [12]:
df['prop_removed']=(df['length']-df['clean_length'])*100/df['length']
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,length,clean_length,prop_removed
0,ham,Go jurong point crazy Available bugis great...,NaN,NaN,NaN,111,80,27.927928
1,ham,Ok lar Joking oni,NaN,NaN,NaN,29,23,20.689655
2,spam,Free entry wkly comp win FA Cup final tkts ...,NaN,NaN,NaN,155,141,9.032258
3,ham,early hor say,NaN,NaN,NaN,49,19,61.224490
4,ham,Nah I usf lives,NaN,NaN,NaN,61,16,73.770492


In [13]:
df['prop_removed'].describe()

count    5572.000000
mean       32.475735
std        18.089656
min         0.000000
25%        19.082714
50%        32.142857
75%        44.680851
max        95.652174
Name: prop_removed, dtype: float64

In [14]:
df['v1']=df['v1'].apply(lambda x:  1 if x=='spam' else 0)
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,length,clean_length,prop_removed
0,0,Go jurong point crazy Available bugis great...,NaN,NaN,NaN,111,80,27.927928
1,0,Ok lar Joking oni,NaN,NaN,NaN,29,23,20.689655
2,1,Free entry wkly comp win FA Cup final tkts ...,NaN,NaN,NaN,155,141,9.032258
3,0,early hor say,NaN,NaN,NaN,49,19,61.224490
4,0,Nah I usf lives,NaN,NaN,NaN,61,16,73.770492


In [15]:
df['v1'].value_counts()

0    4825
1     747
Name: v1, dtype: int64

In [16]:
tfid=TfidfVectorizer()
model=SVC()
x=tfid.fit_transform(df['v2'])
y=df['v1']


In [17]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=5)

In [18]:
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print('accuracy: ',accuracy_score(y_test,y_pred))

accuracy:  0.9739910313901345
